In [1]:
import os
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from sqlalchemy import create_engine, MetaData, Table, select
import string
import boto3
import pickle

# Collect Track Release Dates

From a previous project, connect to database and create dataframe of track data.

In [2]:
engine_name = 'postgresql://' + str(os.environ['zU']) + ':' + str(os.environ['zP']) + str(os.environ['AWS_PROJECT_KOJAK_EC2'])
engine = create_engine(engine_name)

In [3]:
df = pd.read_sql_query('''SELECT 
                          tracks.album_name,
                          tracks.album_id,
                          tracks.duration_ms,
                          tracks.track_id,
                          tracks.track_name,
                          artists.artist_name,
                          artists.artist_id,
                          lyrics.lyrics
                          FROM tracks
                          JOIN artists ON tracks.artist_id = artists.artist_id
                          JOIN lyrics ON tracks.track_id = lyrics.track_id
                          WHERE tracks.energy IS NOT NULL
                          AND lyrics.lyrics IS NOT NULL
                          AND artists.artist_name != 'Kid Rock'
                          ORDER BY tracks.track_name;''', engine)

Clean up dataframe by removing duplicates, correcting capitalization, and remove tracks that don't have lyrics.

In [4]:
df['track_name'] = df['track_name'].apply(lambda i: string.capwords(i))
df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df.drop(df[df['lyrics'].str.contains('<span')].index, inplace=True)
df.reset_index(drop=True, inplace=True)

Connect to Spotify API and get release dates of tracks by their album release date.

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.environ['SPOTIFY_CLIENT_ID'],
                                                     client_secret=os.environ['SPOTIFY_CLIENT_SECRET'])

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Collect release dates of each track and create a new column.

In [7]:
release_dates = []

for index, row in df.iterrows():
    album_id = row['album_id']
    release_date = spotify.album(album_id)['release_date']
    release_dates.append(release_date)

In [12]:
df['release_date'] = release_dates

In [13]:
df.head()

,album_name,album_id,duration_ms,track_id,track_name,artist_name,artist_id,lyrics,release_date
0,Overcast! (20 Year Anniversary Remaster),2fvyHrHdHKlPsq9pJaWIPU,139280,6reu6Jf5DWcRvM0YZyGbzv,@,Atmosphere,1GAS0rb4L8VTPvizAx2O9J,And now with the following collection of ghoul...,2017-10-27
1,Skool Luv Affair,5dht9IOTePdtkmeDegHxG6,239826,1P4fAbjYQqI5jA3Yshgpof,하루만,Bangtan Boys,3Nrfpe0tUJi4K4DXYWgMUX,하루만 내게 시간이 있다면\n달콤한 니 향기에 취해서\n곤히 난 잠들고파\n빡빡한 ...,2015-03-18
2,WAKE UP 通常盤,7bypIG17C3n1BlftyaSJdE,232466,12ZzMA3m2ingQEzmbw3kNR,いいね!,Bangtan Boys,3Nrfpe0tUJi4K4DXYWgMUX,"WANNA BE LOVED…\n\nDON’T WANNA BE FOOL, WANNA ...",2014-12-24
3,Skool Luv Affair,5dht9IOTePdtkmeDegHxG6,240266,7df9Iv1N1EJW1qtp5B98Hk,어디에서 왔는지,Bangtan Boys,3Nrfpe0tUJi4K4DXYWgMUX,가시나야 니는 어데서 왔노\n까리뽕쌈하네 지금 어데로 가노\n니는 몇살이고 니가 내...,2015-03-18
4,Skool Luv Affair,5dht9IOTePdtkmeDegHxG6,238573,2zXhxAKfMj2MieiRUNwaK9,등골브레이커,Bangtan Boys,3Nrfpe0tUJi4K4DXYWgMUX,La la la la la la la la la\nLa la la la la la ...,2015-03-18


In [14]:
with open('tracks_df.pickle', 'wb') as f:
    pickle.dump(df, f)